# Boundary Value Problems

In [ ]:
import numpy as np
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve
import pandas as pd
from sympy import symbols, sympify, lambdify
import ipywidgets as widgets
from IPython.display import display

import matplotlib.pyplot as plt

def solve_bvp():
    # Get user inputs through widgets
    equation_input = widgets.Text(
        value='y\'\'(x) + y(x) = 0',
        description='ODE (in terms of y\'\'(x), y\'(x), y(x) and x):',
        style={'description_width': 'initial'},
        layout={'width': '700px'}
    )

    x_range = widgets.FloatRangeSlider(
        value=[0, 1],
        min=0,
        max=10,
        step=0.1,
        description='x range:',
        style={'description_width': 'initial'}
    )

    left_bc = widgets.Text(
        value='y(0) = 0',
        description='Left boundary condition:',
        style={'description_width': 'initial'}
    )

    right_bc = widgets.Text(
        value='y(1) = 0',
        description='Right boundary condition:',
        style={'description_width': 'initial'}
    )

    step_size = widgets.FloatSlider(
        value=0.1,
        min=0.01,
        max=0.5,
        step=0.01,
        description='Step size:',
        style={'description_width': 'initial'}
    )

    solve_button = widgets.Button(
        description='Solve',
        button_style='success'
    )

    output = widgets.Output()

    display(equation_input, x_range, left_bc, right_bc, step_size, solve_button, output)

    def on_solve_button_clicked(b):
        with output:
            output.clear_output()

            try:
                # Parse the ODE
                eq_str = equation_input.value

                # Extract boundary conditions
                x_min, x_max = x_range.value
                y_left = float(left_bc.value.split('=')[1].strip())
                y_right = float(right_bc.value.split('=')[1].strip())
                h = step_size.value

                # Generate grid points
                x = np.arange(x_min, x_max+h, h)
                n = len(x)

                # Parse equation to handle different forms
                # For simplicity, we'll focus on the form y'' + p(x)y' + q(x)y = f(x)
                # Users need to arrange their equation in this form

                # Example approach for a simple case (can be expanded for more complex equations)
                if "y''(x)" in eq_str:
                    parts = eq_str.split("=")
                    lhs = parts[0].strip()
                    rhs = parts[1].strip() if len(parts) > 1 else "0"

                    # Extract coefficients (simplified for demo)
                    # In practice, would need more robust parsing
                    p_val = 0  # coefficient of y'
                    q_val = 0  # coefficient of y
                    f_val = float(rhs)  # right hand side

                    if "y'(x)" in lhs:
                        p_val = 1  # simplified; should extract actual coefficient
                    if "y(x)" in lhs:
                        q_val = 1  # simplified; should extract actual coefficient
                else:
                    # Default to a simple equation for demonstration
                    p_val, q_val, f_val = 0, 1, 0  # representing y'' + y = 0

                # Set up the system: -y_{i-1} + 2y_i - y_{i+1} = h^2 * (f_i - p_i*dy/dx - q_i*y_i)
                # For simplicity, we'll use constant coefficients

                # Create the diagonal matrices for the finite difference scheme
                main_diag = np.ones(n-2) * (2 + h**2 * q_val)
                off_diag = np.ones(n-3) * (-1 - h * p_val/2)

                # Create the tridiagonal matrix
                A = diags([off_diag, main_diag, off_diag], [-1, 0, 1], shape=(n-2, n-2)).toarray()

                # Set up the right hand side
                b = np.ones(n-2) * h**2 * f_val

                # Apply boundary conditions
                b[0] += (1 + h * p_val/2) * y_left
                b[-1] += (1 + h * p_val/2) * y_right

                # Solve the system
                y_interior = np.linalg.solve(A, b)

                # Combine with boundary values
                y = np.zeros(n)
                y[0] = y_left
                y[1:-1] = y_interior
                y[-1] = y_right

                # Plot the solution
                plt.figure(figsize=(10, 6))
                plt.plot(x, y, 'b-o', label='Numerical Solution')
                plt.grid(True)
                plt.xlabel('x')
                plt.ylabel('y(x)')
                plt.title('Solution of the Boundary Value Problem')
                plt.legend()
                plt.show()

                # Display solution in table format
                df = pd.DataFrame({'x': x, 'y(x)': y})
                display(df)

            except Exception as e:
                print(f"Error: {str(e)}")
                print("Please check your inputs and try again.")

    solve_button.on_click(on_solve_button_clicked)

# Run the function to display the interface
solve_bvp()

In [ ]:
import numpy as np
from scipy.sparse import diags
import matplotlib.pyplot as plt
import pandas as pd

def solve_bvp_simplified(equation_str, x_min, x_max, y_left, y_right, h):
    """
    Solves a boundary value problem using the finite difference method.

    Args:
        equation_str: The ODE as a string (e.g., "y''(x) + y(x) = 0").
        x_min: The minimum value of x.
        x_max: The maximum value of x.
        y_left: The boundary condition at x_min (y(x_min)).
        y_right: The boundary condition at x_max (y(x_max)).
        h: The step size.

    Returns:
        A pandas DataFrame containing the solution (x and y values).
    """
    # Generate grid points
    x = np.arange(x_min, x_max + h, h)
    n = len(x)

    # Assume equation is in the form y'' + p(x)y' + q(x)y = f(x)
    # Simplified coefficient extraction (for demonstration)
    p_val, q_val, f_val = 0, 1, 0  # Default to y'' + y = 0

    # Create tridiagonal matrix
    main_diag = np.ones(n - 2) * (2 + h**2 * q_val)
    off_diag = np.ones(n - 3) * (-1 - h * p_val / 2)
    A = diags([off_diag, main_diag, off_diag], [-1, 0, 1], shape=(n - 2, n - 2)).toarray()

    # Set up right-hand side
    b = np.ones(n - 2) * h**2 * f_val
    b[0] += (1 + h * p_val / 2) * y_left
    b[-1] += (1 + h * p_val / 2) * y_right

    # Solve the system
    y_interior = np.linalg.solve(A, b)

    # Combine with boundary values
    y = np.zeros(n)
    y[0] = y_left
    y[1:-1] = y_interior
    y[-1] = y_right

    # Create and return DataFrame
    df = pd.DataFrame({'x': x, 'y(x)': y})
    return df

# Get inputs from the user
equation = input("Enter the ODE (e.g., y''(x) + y(x) = 0): ")
x_min = float(input("Enter the minimum value of x: "))
x_max = float(input("Enter the maximum value of x: "))
y_left = float(input("Enter the boundary condition at x_min (y(x_min)): "))
y_right = float(input("Enter the boundary condition at x_max (y(x_max)): "))
h = float(input("Enter the step size: "))

# Solve the BVP
solution_df = solve_bvp_simplified(equation, x_min, x_max, y_left, y_right, h)

# Plot the solution
plt.figure(figsize=(10, 6))
plt.plot(solution_df['x'], solution_df['y(x)'], 'b-o', label='Numerical Solution')
plt.grid(True)
plt.xlabel('x')
plt.ylabel('y(x)')
plt.title('Solution of the Boundary Value Problem')
plt.legend()
plt.show()

# Display solution in table format
print(solution_df)